In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pymongo
import datetime

from sqlalchemy import create_engine
import json
from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)

In [2]:
weather_df =pd.read_csv('Resources/US_WeatherEvents_2016-2019.csv')

In [13]:
weather_df.head()

EventId  Type Severity       StartTime(UTC)         EndTime(UTC)  \
0     W-1  Snow    Light  2016-01-06 23:14:00  2016-01-07 00:34:00   
1     W-2  Snow    Light  2016-01-07 04:14:00  2016-01-07 04:54:00   
2     W-3  Snow    Light  2016-01-07 05:54:00  2016-01-07 15:34:00   
3     W-4  Snow    Light  2016-01-08 05:34:00  2016-01-08 05:54:00   
4     W-5  Snow    Light  2016-01-08 13:54:00  2016-01-08 15:54:00   

      TimeZone AirportCode  LocationLat  LocationLng      City    County  \
0  US/Mountain        K04V      38.0972    -106.1689  Saguache  Saguache   
1  US/Mountain        K04V      38.0972    -106.1689  Saguache  Saguache   
2  US/Mountain        K04V      38.0972    -106.1689  Saguache  Saguache   
3  US/Mountain        K04V      38.0972    -106.1689  Saguache  Saguache   
4  US/Mountain        K04V      38.0972    -106.1689  Saguache  Saguache   

  State  ZipCode  
0    CO  81149.0  
1    CO  81149.0  
2    CO  81149.0  
3    CO  81149.0  
4    CO  81149.0

In [3]:
filt = (weather_df["StartTime(UTC)"].str.contains('2018-01-01'))
selected_df = weather_df.loc[filt]

In [4]:
selected_df

EventId  Type Severity       StartTime(UTC)         EndTime(UTC)  \
6225        W-6226  Cold   Severe  2018-01-01 23:57:00  2018-01-02 14:38:00   
11786      W-11789  Cold   Severe  2018-01-01 02:35:00  2018-01-01 03:15:00   
11787      W-11790  Cold   Severe  2018-01-01 03:35:00  2018-01-01 15:15:00   
17998      W-18007  Snow    Light  2018-01-01 01:53:00  2018-01-01 07:53:00   
17999      W-18008  Snow    Light  2018-01-01 11:53:00  2018-01-01 19:53:00   
...            ...   ...      ...                  ...                  ...   
5058813  W-5059492   Fog   Severe  2018-01-01 17:11:00  2018-01-01 17:32:00   
5058814  W-5059493   Fog   Severe  2018-01-01 17:42:00  2018-01-01 17:50:00   
5058815  W-5059494   Fog   Severe  2018-01-01 18:30:00  2018-01-01 19:00:00   
5058816  W-5059495  Snow    Light  2018-01-01 19:14:00  2018-01-01 19:51:00   
5058817  W-5059496   Fog   Severe  2018-01-01 19:51:00  2018-01-01 19:53:00   

            TimeZone AirportCode  LocationLat  LocationLng          City  \
6225      US/Central        KAEL      43.6822     -93.3722    Albert Lea   
11786     US/Central        KCMY      43.9581     -90.7397        Sparta   
11787     US/Central        KCMY      43.9581     -90.7397        Sparta   
17998     US/Eastern        KGRR      42.8808     -85.5228  Grand Rapids   
17999     US/Eastern        KGRR      42.8808     -85.5228  Grand Rapids   
...              ...         ...          ...          ...           ...   
5058813  US/Mountain        KBVR      42.5833    -108.2833        Lander   
5058814  US/Mountain        KBVR      42.5833    -108.2833        Lander   
5058815  US/Mountain        KBVR      42.5833    -108.2833        Lander   
5058816  US/Mountain        KBVR      42.5833    -108.2833        Lander   
5058817  US/Mountain        KBVR      42.5833    -108.2833        Lander   

           County State  ZipCode  
6225     Freeborn    MN  56007.0  
11786      Monroe    WI  54656.0  
11787      Monroe    WI  54656.0  
17998        Kent    MI  49512.0  
17999        Kent    MI  49512.0  
...           ...   ...      ...  
5058813   Fremont    WY  82520.0  
5058814   Fremont    WY  82520.0  
5058815   Fremont    WY  82520.0  
5058816   Fremont    WY  82520.0  
5058817   Fremont    WY  82520.0  

[2149 rows x 13 columns]

In [6]:
selected_df2 = selected_df[['Type','Severity','LocationLat','LocationLng']]

In [7]:
selected_df2.to_csv('Clean/weather_cleaned.csv',index=False)

In [8]:
pd.read_csv('Clean/weather_cleaned.csv')

Type Severity  LocationLat  LocationLng
0     Cold   Severe      43.6822     -93.3722
1     Cold   Severe      43.9581     -90.7397
2     Cold   Severe      43.9581     -90.7397
3     Snow    Light      42.8808     -85.5228
4     Snow    Light      42.8808     -85.5228
...    ...      ...          ...          ...
2144   Fog   Severe      42.5833    -108.2833
2145   Fog   Severe      42.5833    -108.2833
2146   Fog   Severe      42.5833    -108.2833
2147  Snow    Light      42.5833    -108.2833
2148   Fog   Severe      42.5833    -108.2833

[2149 rows x 4 columns]

In [12]:
selected_df2['Type'].unique()

array(['Cold', 'Snow', 'Fog', 'Rain', 'Storm', 'Precipitation', 'Hail'],
      dtype=object)

In [14]:
filt = (weather_df["StartTime(UTC)"].str.contains('2018-01-01'))&(weather_df['Severity']=='Severe')
severe_df = weather_df.loc[filt]

In [15]:
severe_df.columns

Index(['EventId', 'Type', 'Severity', 'StartTime(UTC)', 'EndTime(UTC)',
       'TimeZone', 'AirportCode', 'LocationLat', 'LocationLng', 'City',
       'County', 'State', 'ZipCode'],
      dtype='object')

In [16]:
severe_df2 = severe_df[['Type','LocationLat','LocationLng']]

In [17]:
severe_df2['Type'].unique()

array(['Cold', 'Fog', 'Storm'], dtype=object)

In [24]:
severe_df3=severe_df2.dropna()

In [34]:
severe_df3['Type'].unique()

array(['Cold', 'Fog', 'Storm'], dtype=object)

In [25]:
severe_df3.to_csv('Clean/severe_weather_cleaned2.csv',index=False)

In [26]:
severe_df3 = pd.read_csv('Clean/severe_weather_cleaned2.csv')

In [27]:
severe_df3

Type  LocationLat  LocationLng
0    Cold      43.6822     -93.3722
1    Cold      43.9581     -90.7397
2    Cold      43.9581     -90.7397
3     Fog      45.7691    -122.8637
4    Cold      46.0244     -92.8991
..    ...          ...          ...
732   Fog      42.5833    -108.2833
733   Fog      42.5833    -108.2833
734   Fog      42.5833    -108.2833
735   Fog      42.5833    -108.2833
736   Fog      42.5833    -108.2833

[737 rows x 3 columns]

Type  LocationLat  LocationLng
0    Cold      43.7450    -111.0912
1    Cold      43.7450    -111.0912
2    Cold      43.7450    -111.0912
3    Cold      41.3509     -71.8054
4    Cold      33.2338    -119.4559
..    ...          ...          ...
426   Fog      28.9444     -81.9701
427   Fog      28.9444     -81.9701
428  Cold      37.9740     -92.6910
429  Cold      40.6000    -116.8667
430  Cold      40.6000    -116.8667

[431 rows x 3 columns]

In [4]:
df = pd.read_csv('Resources/2018_select.csv')
df.head()

FL_DATE OP_CARRIER  OP_CARRIER_FL_NUM ORIGIN DEST  CRS_DEP_TIME  \
0  2018-01-01         UA               2429    EWR  DEN          1517   
1  2018-01-01         UA               2427    LAS  SFO          1115   
2  2018-01-01         UA               2426    SNA  DEN          1335   
3  2018-01-01         UA               2425    RSW  ORD          1546   
4  2018-01-01         UA               2424    ORD  ALB           630   

   DEP_TIME  DEP_DELAY  TAXI_OUT  WHEELS_OFF  ...  CRS_ELAPSED_TIME  \
0    1512.0       -5.0      15.0      1527.0  ...             268.0   
1    1107.0       -8.0      11.0      1118.0  ...              99.0   
2    1330.0       -5.0      15.0      1345.0  ...             134.0   
3    1552.0        6.0      19.0      1611.0  ...             190.0   
4     650.0       20.0      13.0       703.0  ...             112.0   

   ACTUAL_ELAPSED_TIME  AIR_TIME  DISTANCE  CARRIER_DELAY  WEATHER_DELAY  \
0                250.0     225.0    1605.0            NaN            NaN   
1                 83.0      65.0     414.0            NaN            NaN   
2                126.0     106.0     846.0            NaN            NaN   
3                182.0     157.0    1120.0            NaN            NaN   
4                106.0      83.0     723.0            NaN            NaN   

  NAS_DELAY  SECURITY_DELAY  LATE_AIRCRAFT_DELAY  Unnamed: 27  
0       NaN             NaN                  NaN          NaN  
1       NaN             NaN                  NaN          NaN  
2       NaN             NaN                  NaN          NaN  
3       NaN             NaN                  NaN          NaN  
4       NaN             NaN                  NaN          NaN  

[5 rows x 28 columns]

## Airport Dataclean

In [5]:
ori_gp = df.groupby(['ORIGIN'])

In [6]:
delay_ds = ori_gp['DEP_DELAY'].mean()
delay_ds
delay_df= pd.DataFrame(delay_ds)
delay_df

DEP_DELAY
ORIGIN            
ABE       7.833333
ABI     209.333333
ABQ      12.779412
ABR       0.500000
ABY      -2.500000
...            ...
VPS      26.071429
WRG            NaN
XNA      38.653846
YNG      15.000000
YUM      -7.750000

[326 rows x 1 columns]

In [7]:
flight_ds=ori_gp['FL_DATE'].count()
flight_ds
flight_df= pd.DataFrame(flight_ds)

In [8]:
cancel_ds=ori_gp['CANCELLED'].sum()
cancel_ds
cancel_df= pd.DataFrame(cancel_ds)

In [9]:
airport_df = pd.read_csv('Resources/completed_airports.csv')

In [10]:
airport_df.head()
airport_df = airport_df[['IATA','Airport','City','STATE','LATITUDE','LONGITUDE']]

In [11]:
merged_df = pd.merge(flight_df,delay_df,left_index=True, right_index=True, how='inner')

In [12]:
merged_df = pd.merge(merged_df,cancel_df,left_index=True, right_index=True, how='inner')

In [13]:
merged_df.reset_index(inplace = True)

In [14]:
merged_df

ORIGIN  FL_DATE   DEP_DELAY  CANCELLED
0      ABE        6    7.833333        0.0
1      ABI        6  209.333333        0.0
2      ABQ       68   12.779412        0.0
3      ABR        2    0.500000        0.0
4      ABY        2   -2.500000        0.0
..     ...      ...         ...        ...
321    VPS       14   26.071429        0.0
322    WRG        2         NaN        2.0
323    XNA       28   38.653846        2.0
324    YNG        1   15.000000        0.0
325    YUM        4   -7.750000        0.0

[326 rows x 4 columns]

In [15]:
air_df= pd.merge(merged_df,airport_df,left_on='ORIGIN',right_on='IATA',how='left')

In [16]:
air_df = air_df.dropna(how='any')

In [17]:
air_df = air_df.drop(columns=['IATA'])
air_df

ORIGIN  FL_DATE   DEP_DELAY  CANCELLED  \
0      ABE        6    7.833333        0.0   
1      ABI        6  209.333333        0.0   
2      ABQ       68   12.779412        0.0   
3      ABR        2    0.500000        0.0   
4      ABY        2   -2.500000        0.0   
..     ...      ...         ...        ...   
317    TYS       37    3.416667        1.0   
320    VLD        2   11.000000        0.0   
321    VPS       14   26.071429        0.0   
323    XNA       28   38.653846        2.0   
325    YUM        4   -7.750000        0.0   

                                               Airport               City  \
0    Lehigh Valley International Airport (was Allen...          Allentown   
1                             Abilene Regional Airport            Abilene   
2                    Albuquerque International Sunport        Albuquerque   
3                            Aberdeen Regional Airport           Aberdeen   
4                   Southwest Georgia Regional Airport             Albany   
..                                                 ...                ...   
317                               McGhee Tyson Airport          Knoxville   
320                          Valdosta Regional Airport           Valdosta   
321  Destin–Fort Walton Beach Airport / Eglin Air F...  Fort Walton Beach   
323                Northwest Arkansas National Airport       Fayetteville   
325             Yuma International Airport / MCAS Yuma               Yuma   

    STATE  LATITUDE  LONGITUDE  
0      PA  40.65236  -75.44040  
1      TX  32.41132  -99.68190  
2      NM  35.04022 -106.60919  
3      SD  45.44906  -98.42183  
4      GA  31.53552  -84.19447  
..    ...       ...        ...  
317    TN  35.81249  -83.99286  
320    GA  30.78250  -83.27672  
321    FL  30.48325  -86.52540  
323    AR  36.28187  -94.30681  
325    AZ  32.65658 -114.60597  

[294 rows x 9 columns]

In [18]:
air_df.columns=['IATA_code','Flights','AvgDelay','Cancelled','Fullname','City','State','Lat','Lon']

In [19]:
air_df

IATA_code  Flights    AvgDelay  Cancelled  \
0         ABE        6    7.833333        0.0   
1         ABI        6  209.333333        0.0   
2         ABQ       68   12.779412        0.0   
3         ABR        2    0.500000        0.0   
4         ABY        2   -2.500000        0.0   
..        ...      ...         ...        ...   
317       TYS       37    3.416667        1.0   
320       VLD        2   11.000000        0.0   
321       VPS       14   26.071429        0.0   
323       XNA       28   38.653846        2.0   
325       YUM        4   -7.750000        0.0   

                                              Fullname               City  \
0    Lehigh Valley International Airport (was Allen...          Allentown   
1                             Abilene Regional Airport            Abilene   
2                    Albuquerque International Sunport        Albuquerque   
3                            Aberdeen Regional Airport           Aberdeen   
4                   Southwest Georgia Regional Airport             Albany   
..                                                 ...                ...   
317                               McGhee Tyson Airport          Knoxville   
320                          Valdosta Regional Airport           Valdosta   
321  Destin–Fort Walton Beach Airport / Eglin Air F...  Fort Walton Beach   
323                Northwest Arkansas National Airport       Fayetteville   
325             Yuma International Airport / MCAS Yuma               Yuma   

    State       Lat        Lon  
0      PA  40.65236  -75.44040  
1      TX  32.41132  -99.68190  
2      NM  35.04022 -106.60919  
3      SD  45.44906  -98.42183  
4      GA  31.53552  -84.19447  
..    ...       ...        ...  
317    TN  35.81249  -83.99286  
320    GA  30.78250  -83.27672  
321    FL  30.48325  -86.52540  
323    AR  36.28187  -94.30681  
325    AZ  32.65658 -114.60597  

[294 rows x 9 columns]

In [20]:
air_df.to_csv('Clean/airport_cleaned.csv',index=False)

In [20]:
air_df = pd.read_csv('Clean/airport_cleaned.csv')

In [22]:
air_na_df=air_df.fillna('null')

In [24]:
air_na_df.to_csv('Clean/airport_na_cleaned.csv',index=False)

In [25]:
pd.read_csv('Clean/airport_na_cleaned.csv')

IATA_code  Flights    AvgDelay  Cancelled  \
0         ABE        6    7.833333        0.0   
1         ABI        6  209.333333        0.0   
2         ABQ       68   12.779412        0.0   
3         ABR        2    0.500000        0.0   
4         ABY        2   -2.500000        0.0   
..        ...      ...         ...        ...   
289       TYS       37    3.416667        1.0   
290       VLD        2   11.000000        0.0   
291       VPS       14   26.071429        0.0   
292       XNA       28   38.653846        2.0   
293       YUM        4   -7.750000        0.0   

                                              Fullname               City  \
0    Lehigh Valley International Airport (was Allen...          Allentown   
1                             Abilene Regional Airport            Abilene   
2                    Albuquerque International Sunport        Albuquerque   
3                            Aberdeen Regional Airport           Aberdeen   
4                   Southwest Georgia Regional Airport             Albany   
..                                                 ...                ...   
289                               McGhee Tyson Airport          Knoxville   
290                          Valdosta Regional Airport           Valdosta   
291  Destin–Fort Walton Beach Airport / Eglin Air F...  Fort Walton Beach   
292                Northwest Arkansas National Airport       Fayetteville   
293             Yuma International Airport / MCAS Yuma               Yuma   

    State       Lat        Lon  
0      PA  40.65236  -75.44040  
1      TX  32.41132  -99.68190  
2      NM  35.04022 -106.60919  
3      SD  45.44906  -98.42183  
4      GA  31.53552  -84.19447  
..    ...       ...        ...  
289    TN  35.81249  -83.99286  
290    GA  30.78250  -83.27672  
291    FL  30.48325  -86.52540  
292    AR  36.28187  -94.30681  
293    AZ  32.65658 -114.60597  

[294 rows x 9 columns]

# Flight cleanup

In [27]:
flight_df = df.copy()
flight_df

FL_DATE OP_CARRIER  OP_CARRIER_FL_NUM ORIGIN DEST  CRS_DEP_TIME  \
0      2018-01-01         UA               2429    EWR  DEN          1517   
1      2018-01-01         UA               2427    LAS  SFO          1115   
2      2018-01-01         UA               2426    SNA  DEN          1335   
3      2018-01-01         UA               2425    RSW  ORD          1546   
4      2018-01-01         UA               2424    ORD  ALB           630   
...           ...        ...                ...    ...  ...           ...   
17909  2018-01-01         DL               2988    SJC  MSP           620   
17910  2018-01-01         DL               2989    SEA  ORD          1802   
17911  2018-01-01         DL               2990    ATL  DCA          1005   
17912  2018-01-01         DL               2999    ATL  SYR           908   
17913  2018-01-01         DL               2999    SYR  ATL          1150   

       DEP_TIME  DEP_DELAY  TAXI_OUT  WHEELS_OFF  ...  CRS_ELAPSED_TIME  \
0        1512.0       -5.0      15.0      1527.0  ...             268.0   
1        1107.0       -8.0      11.0      1118.0  ...              99.0   
2        1330.0       -5.0      15.0      1345.0  ...             134.0   
3        1552.0        6.0      19.0      1611.0  ...             190.0   
4         650.0       20.0      13.0       703.0  ...             112.0   
...         ...        ...       ...         ...  ...               ...   
17909     621.0        1.0      16.0       637.0  ...             235.0   
17910    1754.0       -8.0      12.0      1806.0  ...             237.0   
17911    1012.0        7.0      23.0      1035.0  ...              99.0   
17912     903.0       -5.0      20.0       923.0  ...             127.0   
17913    1150.0        0.0      14.0      1204.0  ...             162.0   

       ACTUAL_ELAPSED_TIME  AIR_TIME  DISTANCE  CARRIER_DELAY  WEATHER_DELAY  \
0                    250.0     225.0    1605.0            NaN            NaN   
1                     83.0      65.0     414.0            NaN            NaN   
2                    126.0     106.0     846.0            NaN            NaN   
3                    182.0     157.0    1120.0            NaN            NaN   
4                    106.0      83.0     723.0            NaN            NaN   
...                    ...       ...       ...            ...            ...   
17909                219.0     199.0    1575.0            NaN            NaN   
17910                224.0     203.0    1721.0            NaN            NaN   
17911                 95.0      70.0     547.0            NaN            NaN   
17912                128.0     101.0     794.0            NaN            NaN   
17913                146.0     124.0     794.0            NaN            NaN   

      NAS_DELAY  SECURITY_DELAY  LATE_AIRCRAFT_DELAY  Unnamed: 27  
0           NaN             NaN                  NaN          NaN  
1           NaN             NaN                  NaN          NaN  
2           NaN             NaN                  NaN          NaN  
3           NaN             NaN                  NaN          NaN  
4           NaN             NaN                  NaN          NaN  
...         ...             ...                  ...          ...  
17909       NaN             NaN                  NaN          NaN  
17910       NaN             NaN                  NaN          NaN  
17911       NaN             NaN                  NaN          NaN  
17912       NaN             NaN                  NaN          NaN  
17913       NaN             NaN                  NaN          NaN  

[17914 rows x 28 columns]

In [28]:
flight_df['DEP_TIME'] = flight_df['DEP_TIME'].loc[~flight_df['DEP_TIME'].isna()].astype(int).astype(str)
flight_df['DEP_TIME']

0        1512
1        1107
2        1330
3        1552
4         650
         ... 
17909     621
17910    1754
17911    1012
17912     903
17913    1150
Name: DEP_TIME, Length: 17914, dtype: object

In [29]:
flight_df['DEP_TIME'].loc[flight_df['DEP_TIME'].str.len()==3] = '0'+flight_df['DEP_TIME'].loc[flight_df['DEP_TIME'].str.len()==3]


C:\Users\Wei\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [30]:
flight_df['DEP_TIME'] = flight_df['DEP_TIME'].str[0:2]+':'+flight_df['DEP_TIME'].str[2:4]

In [31]:
flight_df['DEP_TIME']

0        15:12
1        11:07
2        13:30
3        15:52
4        06:50
         ...  
17909    06:21
17910    17:54
17911    10:12
17912    09:03
17913    11:50
Name: DEP_TIME, Length: 17914, dtype: object

In [32]:
flight_df=flight_df[['FL_DATE','OP_CARRIER','OP_CARRIER_FL_NUM','ORIGIN', 'DEST','AIR_TIME','DISTANCE','DEP_TIME', 'DEP_DELAY','CANCELLED']]

In [33]:
flight_df

FL_DATE OP_CARRIER  OP_CARRIER_FL_NUM ORIGIN DEST  AIR_TIME  \
0      2018-01-01         UA               2429    EWR  DEN     225.0   
1      2018-01-01         UA               2427    LAS  SFO      65.0   
2      2018-01-01         UA               2426    SNA  DEN     106.0   
3      2018-01-01         UA               2425    RSW  ORD     157.0   
4      2018-01-01         UA               2424    ORD  ALB      83.0   
...           ...        ...                ...    ...  ...       ...   
17909  2018-01-01         DL               2988    SJC  MSP     199.0   
17910  2018-01-01         DL               2989    SEA  ORD     203.0   
17911  2018-01-01         DL               2990    ATL  DCA      70.0   
17912  2018-01-01         DL               2999    ATL  SYR     101.0   
17913  2018-01-01         DL               2999    SYR  ATL     124.0   

       DISTANCE DEP_TIME  DEP_DELAY  CANCELLED  
0        1605.0    15:12       -5.0        0.0  
1         414.0    11:07       -8.0        0.0  
2         846.0    13:30       -5.0        0.0  
3        1120.0    15:52        6.0        0.0  
4         723.0    06:50       20.0        0.0  
...         ...      ...        ...        ...  
17909    1575.0    06:21        1.0        0.0  
17910    1721.0    17:54       -8.0        0.0  
17911     547.0    10:12        7.0        0.0  
17912     794.0    09:03       -5.0        0.0  
17913     794.0    11:50        0.0        0.0  

[17914 rows x 10 columns]

In [34]:
flight_df.columns=['Date','Airline','Flight_Num','Origin','Destination','Air_Time','Distance','Dep_Time','Delay', 'Cancelled']

In [35]:
flight_df

Date Airline  Flight_Num Origin Destination  Air_Time  Distance  \
0      2018-01-01      UA        2429    EWR         DEN     225.0    1605.0   
1      2018-01-01      UA        2427    LAS         SFO      65.0     414.0   
2      2018-01-01      UA        2426    SNA         DEN     106.0     846.0   
3      2018-01-01      UA        2425    RSW         ORD     157.0    1120.0   
4      2018-01-01      UA        2424    ORD         ALB      83.0     723.0   
...           ...     ...         ...    ...         ...       ...       ...   
17909  2018-01-01      DL        2988    SJC         MSP     199.0    1575.0   
17910  2018-01-01      DL        2989    SEA         ORD     203.0    1721.0   
17911  2018-01-01      DL        2990    ATL         DCA      70.0     547.0   
17912  2018-01-01      DL        2999    ATL         SYR     101.0     794.0   
17913  2018-01-01      DL        2999    SYR         ATL     124.0     794.0   

      Dep_Time  Delay  Cancelled  
0        15:12   -5.0        0.0  
1        11:07   -8.0        0.0  
2        13:30   -5.0        0.0  
3        15:52    6.0        0.0  
4        06:50   20.0        0.0  
...        ...    ...        ...  
17909    06:21    1.0        0.0  
17910    17:54   -8.0        0.0  
17911    10:12    7.0        0.0  
17912    09:03   -5.0        0.0  
17913    11:50    0.0        0.0  

[17914 rows x 10 columns]

In [36]:
flight_df['Cancelled']=flight_df['Cancelled'].replace(0, 'No')

<ipython-input-36-add50f840b2c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flight_df['Cancelled']=flight_df['Cancelled'].replace(0, 'No')


In [37]:
flight_df['Cancelled']=flight_df['Cancelled'].replace(1, 'Yes')

<ipython-input-37-e87dd71ed1d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flight_df['Cancelled']=flight_df['Cancelled'].replace(1, 'Yes')


In [38]:
flight_df.to_csv('Clean/flights_cleaned.csv',index=False)

In [39]:
pd.read_csv('Clean/flights_cleaned.csv')

Date Airline  Flight_Num Origin Destination  Air_Time  Distance  \
0      2018-01-01      UA        2429    EWR         DEN     225.0    1605.0   
1      2018-01-01      UA        2427    LAS         SFO      65.0     414.0   
2      2018-01-01      UA        2426    SNA         DEN     106.0     846.0   
3      2018-01-01      UA        2425    RSW         ORD     157.0    1120.0   
4      2018-01-01      UA        2424    ORD         ALB      83.0     723.0   
...           ...     ...         ...    ...         ...       ...       ...   
17909  2018-01-01      DL        2988    SJC         MSP     199.0    1575.0   
17910  2018-01-01      DL        2989    SEA         ORD     203.0    1721.0   
17911  2018-01-01      DL        2990    ATL         DCA      70.0     547.0   
17912  2018-01-01      DL        2999    ATL         SYR     101.0     794.0   
17913  2018-01-01      DL        2999    SYR         ATL     124.0     794.0   

      Dep_Time  Delay Cancelled  
0        15:12   -5.0        No  
1        11:07   -8.0        No  
2        13:30   -5.0        No  
3        15:52    6.0        No  
4        06:50   20.0        No  
...        ...    ...       ...  
17909    06:21    1.0        No  
17910    17:54   -8.0        No  
17911    10:12    7.0        No  
17912    09:03   -5.0        No  
17913    11:50    0.0        No  

[17914 rows x 10 columns]

In [40]:
flight_df

Date Airline  Flight_Num Origin Destination  Air_Time  Distance  \
0      2018-01-01      UA        2429    EWR         DEN     225.0    1605.0   
1      2018-01-01      UA        2427    LAS         SFO      65.0     414.0   
2      2018-01-01      UA        2426    SNA         DEN     106.0     846.0   
3      2018-01-01      UA        2425    RSW         ORD     157.0    1120.0   
4      2018-01-01      UA        2424    ORD         ALB      83.0     723.0   
...           ...     ...         ...    ...         ...       ...       ...   
17909  2018-01-01      DL        2988    SJC         MSP     199.0    1575.0   
17910  2018-01-01      DL        2989    SEA         ORD     203.0    1721.0   
17911  2018-01-01      DL        2990    ATL         DCA      70.0     547.0   
17912  2018-01-01      DL        2999    ATL         SYR     101.0     794.0   
17913  2018-01-01      DL        2999    SYR         ATL     124.0     794.0   

      Dep_Time  Delay Cancelled  
0        15:12   -5.0        No  
1        11:07   -8.0        No  
2        13:30   -5.0        No  
3        15:52    6.0        No  
4        06:50   20.0        No  
...        ...    ...       ...  
17909    06:21    1.0        No  
17910    17:54   -8.0        No  
17911    10:12    7.0        No  
17912    09:03   -5.0        No  
17913    11:50    0.0        No  

[17914 rows x 10 columns]

In [41]:
## Airline info

In [44]:
airline_df = pd.read_csv('Resources/completed_airlines.csv')
airline_df

IATA                                        Airline
0    EVY        34 Squadron, Royal Australian Air Force
1     1A                          Amadeus IT Group S.A.
2     1B  Sabre travel network Asia-Pacific (ex-Abacus)
3     1E                                      Travelsky
4     1F                Infini travel information, Inc.
..   ...                                            ...
972   Y8                          Yangtze River Express
973   Y0             Yellow Air Taxi/Friendship Airways
974   IY                                        Yemenia
975   Q3                                Zambian Airways
976   Z4                                  Zoom Airlines

[977 rows x 2 columns]

In [45]:
flight_extra_df= pd.merge(flight_df,airline_df,left_on='Airline',right_on='IATA', how='left')
flight_extra_df

Date Airline_x  Flight_Num Origin Destination  Air_Time  \
0      2018-01-01        UA        2429    EWR         DEN     225.0   
1      2018-01-01        UA        2427    LAS         SFO      65.0   
2      2018-01-01        UA        2426    SNA         DEN     106.0   
3      2018-01-01        UA        2425    RSW         ORD     157.0   
4      2018-01-01        UA        2424    ORD         ALB      83.0   
...           ...       ...         ...    ...         ...       ...   
17909  2018-01-01        DL        2988    SJC         MSP     199.0   
17910  2018-01-01        DL        2989    SEA         ORD     203.0   
17911  2018-01-01        DL        2990    ATL         DCA      70.0   
17912  2018-01-01        DL        2999    ATL         SYR     101.0   
17913  2018-01-01        DL        2999    SYR         ATL     124.0   

       Distance Dep_Time  Delay Cancelled IATA        Airline_y  
0        1605.0    15:12   -5.0        No   UA  United Airlines  
1         414.0    11:07   -8.0        No   UA  United Airlines  
2         846.0    13:30   -5.0        No   UA  United Airlines  
3        1120.0    15:52    6.0        No   UA  United Airlines  
4         723.0    06:50   20.0        No   UA  United Airlines  
...         ...      ...    ...       ...  ...              ...  
17909    1575.0    06:21    1.0        No   DL  Delta Air Lines  
17910    1721.0    17:54   -8.0        No   DL  Delta Air Lines  
17911     547.0    10:12    7.0        No   DL  Delta Air Lines  
17912     794.0    09:03   -5.0        No   DL  Delta Air Lines  
17913     794.0    11:50    0.0        No   DL  Delta Air Lines  

[17914 rows x 12 columns]

In [47]:
flight_extra_df.drop(columns=['IATA'],inplace= True)

In [48]:
flight_extra_df

Date Airline_x  Flight_Num Origin Destination  Air_Time  \
0      2018-01-01        UA        2429    EWR         DEN     225.0   
1      2018-01-01        UA        2427    LAS         SFO      65.0   
2      2018-01-01        UA        2426    SNA         DEN     106.0   
3      2018-01-01        UA        2425    RSW         ORD     157.0   
4      2018-01-01        UA        2424    ORD         ALB      83.0   
...           ...       ...         ...    ...         ...       ...   
17909  2018-01-01        DL        2988    SJC         MSP     199.0   
17910  2018-01-01        DL        2989    SEA         ORD     203.0   
17911  2018-01-01        DL        2990    ATL         DCA      70.0   
17912  2018-01-01        DL        2999    ATL         SYR     101.0   
17913  2018-01-01        DL        2999    SYR         ATL     124.0   

       Distance Dep_Time  Delay Cancelled        Airline_y  
0        1605.0    15:12   -5.0        No  United Airlines  
1         414.0    11:07   -8.0        No  United Airlines  
2         846.0    13:30   -5.0        No  United Airlines  
3        1120.0    15:52    6.0        No  United Airlines  
4         723.0    06:50   20.0        No  United Airlines  
...         ...      ...    ...       ...              ...  
17909    1575.0    06:21    1.0        No  Delta Air Lines  
17910    1721.0    17:54   -8.0        No  Delta Air Lines  
17911     547.0    10:12    7.0        No  Delta Air Lines  
17912     794.0    09:03   -5.0        No  Delta Air Lines  
17913     794.0    11:50    0.0        No  Delta Air Lines  

[17914 rows x 11 columns]

In [50]:
flight_extra_df.columns=['Date','Airline','Flight_Num','Origin','Destination','Air_Time','Distance','Dep_Time','Delay', 'Cancelled', 'Airline_Info']

In [52]:
flight_extra_df.to_csv('Clean/flights__extra_cleaned.csv',index=False)

In [3]:
flight_extra_df = pd.read_csv('Clean/flights__extra_cleaned.csv')

In [11]:
flight_extra_na_df=flight_extra_df.fillna('null')

In [13]:
flight_extra_na_df.to_csv('Clean/flights_extra_na_cleaned.csv',index=False)

In [5]:
flight_extra_na_df = pd.read_csv('Clean/flights_extra_na_cleaned.csv')
flight_extra_na_df

Date Airline  Flight_Num Origin Destination  Air_Time  Distance  \
0      2018-01-01      UA        2429    EWR         DEN     225.0    1605.0   
1      2018-01-01      UA        2427    LAS         SFO      65.0     414.0   
2      2018-01-01      UA        2426    SNA         DEN     106.0     846.0   
3      2018-01-01      UA        2425    RSW         ORD     157.0    1120.0   
4      2018-01-01      UA        2424    ORD         ALB      83.0     723.0   
...           ...     ...         ...    ...         ...       ...       ...   
17909  2018-01-01      DL        2988    SJC         MSP     199.0    1575.0   
17910  2018-01-01      DL        2989    SEA         ORD     203.0    1721.0   
17911  2018-01-01      DL        2990    ATL         DCA      70.0     547.0   
17912  2018-01-01      DL        2999    ATL         SYR     101.0     794.0   
17913  2018-01-01      DL        2999    SYR         ATL     124.0     794.0   

      Dep_Time  Delay Cancelled     Airline_Info  
0        15:12   -5.0        No  United Airlines  
1        11:07   -8.0        No  United Airlines  
2        13:30   -5.0        No  United Airlines  
3        15:52    6.0        No  United Airlines  
4        06:50   20.0        No  United Airlines  
...        ...    ...       ...              ...  
17909    06:21    1.0        No  Delta Air Lines  
17910    17:54   -8.0        No  Delta Air Lines  
17911    10:12    7.0        No  Delta Air Lines  
17912    09:03   -5.0        No  Delta Air Lines  
17913    11:50    0.0        No  Delta Air Lines  

[17914 rows x 11 columns]

In [6]:
flight_extra_na_df['Airline'].unique()

array(['UA', 'AS', '9E', 'B6', 'EV', 'F9', 'G4', 'HA', 'MQ', 'NK', 'OH',
       'OO', 'VX', 'WN', 'YV', 'YX', 'AA', 'DL'], dtype=object)

### mongo database

In [55]:
# database: air_df, flight_extra_df

In [29]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')

In [30]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [31]:
db = client.project2_db

In [59]:
db.airport.insert_many(air_df.to_dict('records'))

In [60]:
db.flight.insert_many(flight_extra_df.to_dict('records'))

In [72]:
listings = db.airport.find()
airport_data=[]
for listing in listings:
    airport_data.append(listing)
print(airport_data[1])

{'_id': ObjectId('600394a97dc54f2bf8bc2deb'), 'IATA_code': 'ABI', 'Flights': 6, 'AvgDelay': 209.33333333333334, 'Cancelled': 0.0, 'Fullname': 'Abilene Regional Airport', 'City': 'Abilene', 'State': 'TX', 'Lat': 32.41132, 'Lon': -99.6819}


In [62]:
db.flight

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'project2_db'), 'flight')

In [ ]:
# database without NA: air_na_df, flight_extra_na_df

In [18]:
db.flight_na.insert_many(flight_extra_na_df.to_dict('records'))

In [26]:
db.airport_na.insert_many(air_na_df.to_dict('records'))

In [32]:
db.severe_180101_na.insert_many(severe_df3.to_dict('records'))

In [ ]:
selected_df2